In [1]:
import tensorflow as tf
from tensorflow.keras.applications import DenseNet121
from tensorflow.keras.layers import Dense, Flatten, Dropout, GlobalAveragePooling2D
from tensorflow.keras.models import Model
from tensorflow.keras.preprocessing.image import ImageDataGenerator
import os

# Paths to your dataset
train_dir = 'Pictures/Train'
val_dir = 'Pictures/Val'
test_dir = 'Pictures/Test'

# Parameters
IMG_SIZE = (224, 224)  # Image dimensions
BATCH_SIZE = 32        # Batch size
EPOCHS = 20            # Number of epochs
NUM_CLASSES = len(os.listdir(train_dir))  # Count classes dynamically

# Data Generators
train_datagen = ImageDataGenerator(
    rescale=1.0 / 255,  # Normalize pixel values
    rotation_range=20,
    width_shift_range=0.2,
    height_shift_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True,
    fill_mode='nearest'
)

val_datagen = ImageDataGenerator(rescale=1.0 / 255)
test_datagen = ImageDataGenerator(rescale=1.0 / 255)

# Load datasets
train_generator = train_datagen.flow_from_directory(
    train_dir,
    target_size=IMG_SIZE,
    batch_size=BATCH_SIZE,
    class_mode='categorical'
)

val_generator = val_datagen.flow_from_directory(
    val_dir,
    target_size=IMG_SIZE,
    batch_size=BATCH_SIZE,
    class_mode='categorical'
)

test_generator = test_datagen.flow_from_directory(
    test_dir,
    target_size=IMG_SIZE,
    batch_size=BATCH_SIZE,
    class_mode='categorical',
    shuffle=False  # Important for evaluation
)

# Load DenseNet121
base_model = DenseNet121(weights='imagenet', include_top=False, input_shape=(224, 224, 3))

# Freeze base model layers
for layer in base_model.layers:
    layer.trainable = False

# Add custom layers
x = base_model.output
x = GlobalAveragePooling2D()(x)  # Better for DenseNet
x = Dense(512, activation='relu')(x)
x = Dropout(0.5)(x)
output = Dense(NUM_CLASSES, activation='softmax')(x)

# Create the model
model = Model(inputs=base_model.input, outputs=output)

# Compile the model
model.compile(
    optimizer=tf.keras.optimizers.Adam(learning_rate=0.001),
    loss='categorical_crossentropy',
    metrics=['accuracy']
)

# Train the model
history = model.fit(
    train_generator,
    epochs=EPOCHS,
    validation_data=val_generator,
    verbose=1
)

# Save the model
model.save('P_densenet121_medical_model_2.h5')

# Evaluate the model on the test set
loss, accuracy = model.evaluate(test_generator)
print(f"Test Accuracy: {accuracy * 100:.2f}%")

# Predict on test data (optional, for detailed evaluation)
predictions = model.predict(test_generator)
predicted_classes = tf.argmax(predictions, axis=1)
true_classes = test_generator.classes

# Print class mapping
class_indices = train_generator.class_indices
class_mapping = {v: k for k, v in class_indices.items()}
print("Class Mapping:", class_mapping)


Found 33522 images belonging to 16 classes.
Found 7505 images belonging to 16 classes.
Found 4634 images belonging to 16 classes.


c:\Users\Anivesh Patel\anaconda3\Lib\site-packages\keras\src\trainers\data_adapters\py_dataset_adapter.py:121: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


Epoch 1/20
1048/1048 ━━━━━━━━━━━━━━━━━━━━ 0s 1s/step - accuracy: 0.5120 - loss: 1.4077

c:\Users\Anivesh Patel\anaconda3\Lib\site-packages\keras\src\trainers\data_adapters\py_dataset_adapter.py:121: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


1048/1048 ━━━━━━━━━━━━━━━━━━━━ 1329s 1s/step - accuracy: 0.5120 - loss: 1.4075 - val_accuracy: 0.6140 - val_loss: 1.0192
Epoch 2/20
1048/1048 ━━━━━━━━━━━━━━━━━━━━ 1153s 1s/step - accuracy: 0.6057 - loss: 1.0491 - val_accuracy: 0.6365 - val_loss: 0.9776
Epoch 3/20
1048/1048 ━━━━━━━━━━━━━━━━━━━━ 1025s 978ms/step - accuracy: 0.6225 - loss: 0.9961 - val_accuracy: 0.6536 - val_loss: 0.9089
Epoch 4/20
1048/1048 ━━━━━━━━━━━━━━━━━━━━ 979s 934ms/step - accuracy: 0.6323 - loss: 0.9602 - val_accuracy: 0.6417 - val_loss: 0.9376
Epoch 5/20
1048/1048 ━━━━━━━━━━━━━━━━━━━━ 975s 930ms/step - accuracy: 0.6378 - loss: 0.9493 - val_accuracy: 0.6502 - val_loss: 0.9211
Epoch 6/20
1048/1048 ━━━━━━━━━━━━━━━━━━━━ 975s 931ms/step - accuracy: 0.6420 - loss: 0.9337 - val_accuracy: 0.6632 - val_loss: 0.8772
Epoch 7/20
1048/1048 ━━━━━━━━━━━━━━━━━━━━ 973s 929ms/step - accuracy: 0.6504 - loss: 0.9129 - val_accuracy: 0.6629 - val_loss: 0.8943
Epoch 8/20
1048/1048 ━━━━━━━━━━━━━━━━━━━━ 971s 927ms/step - accuracy: 0.6581

 11/145 ━━━━━━━━━━━━━━━━━━━━ 1:36 720ms/step - accuracy: 0.5284 - loss: 1.1297

c:\Users\Anivesh Patel\anaconda3\Lib\site-packages\PIL\Image.py:1056: UserWarning: Palette images with Transparency expressed in bytes should be converted to RGBA images
  warnings.warn(


145/145 ━━━━━━━━━━━━━━━━━━━━ 118s 814ms/step - accuracy: 0.6929 - loss: 0.8024
Test Accuracy: 69.83%
145/145 ━━━━━━━━━━━━━━━━━━━━ 101s 684ms/step
Class Mapping: {0: 'Acne', 1: 'Cancer', 2: 'Dermatofibroma', 3: 'Diabetic Foot Ulcers', 4: 'Eczema', 5: 'Melanocytic nevus', 6: 'Melanoma', 7: 'Milia', 8: 'Ordinary wound', 9: 'Pigment', 10: 'Rosacea', 11: 'Vascular lesion', 12: 'carcinoma', 13: 'infection', 14: 'keratosis', 15: 'non-cancer'}
